In [40]:
from transformer4 import Transformer
import time
import tensorflow as tf
import tensorflow_text

In [41]:
tf.__version__

'2.15.0'

In [42]:
# tf.config.set_visible_devices([], 'GPU')

In [43]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [44]:
model_name = 'armenian_tokenizer'
tokenizer = tf.saved_model.load(model_name).am

In [45]:
# import os
# import pickle
# 
# def find_top_and_bottom_pikl(directory, top_n=6):
#     results = []
# 
#     for filename in os.listdir(directory):
#         if filename.endswith('.pkl'):
#             filepath = os.path.join(directory, filename)
#             with open(filepath, 'rb') as f:
#                 data = pickle.load(f)
#                 if 'val_masked_accuracy' in data and 'loss' in data:
#                     accuracy = data['val_masked_accuracy'][-1]
#                     loss = data['loss'][-1]
#                     loss_count = len(data['loss'])  # Count the number of items in the loss list
#                     results.append((filename, accuracy, loss, loss_count))
# 
#     # Sort the results based on accuracy in descending order for top N
#     results.sort(key=lambda x: x[1], reverse=True)
#     top_results = results[:top_n]
# 
#     # Sort the results based on accuracy in ascending order for bottom N
#     results.sort(key=lambda x: x[1])
#     bottom_results = results[:top_n]
# 
#     return top_results, bottom_results
# 
# # Specify your directory here
# directory_path = '/home/vahan/Documents/machine_translation/history'
# 
# top_pikl_files, bottom_pikl_files = find_top_and_bottom_pikl(directory_path)
# if top_pikl_files:
#     print("Top pickle files with highest val_masked_accuracy:")
#     for idx, (filename, accuracy, loss, loss_count) in enumerate(top_pikl_files, start=1):
#         print(f"{idx}. {filename} with val_masked_accuracy of {accuracy}, loss of {loss}, and {loss_count} loss items")
# else:
#     print("No pickle files found or no file contains 'val_masked_accuracy' and 'loss' keys")
# 
# if bottom_pikl_files:
#     print("\nPickle files with lowest val_masked_accuracy:")
#     for idx, (filename, accuracy, loss, loss_count) in enumerate(bottom_pikl_files, start=1):
#         print(f"{idx}. {filename} with val_masked_accuracy of {accuracy}, loss of {loss}, and {loss_count} loss items")
# else:
#     print("No pickle files found or no file contains 'val_masked_accuracy' and 'loss' keys")


In [46]:
# best_indicis = [3, 21, 15, 6, 7, 18]
# worst_indicis = [4, 2, 10, 17, 12, 1]
# print("best_configs")
# for i in best_indicis:
#     with open(f"/home/vahan/Documents/machine_translation/pkls/{i}.pkl", "rb") as f:
#         config = pickle.load(f)
#         print(config)
# 
# 
# print()
# print("worst_configs")
# for i in worst_indicis:
#     with open(f"/home/vahan/Documents/machine_translation/pkls/{i}.pkl", "rb") as f:
#         config = pickle.load(f)
#         print(config)

In [47]:
# with open("/home/vahan/Documents/machine_translation/pkls/3.pkl", "rb") as f:
#     config = pickle.load(f)
# config

In [48]:
# num_layers = config['num_layers']
# d_model = config['d_model']
# dff = config['dff']
# num_heads = config['num_head']
# dropout_rate = config['dropout_rate']
# batch_size = config['batch_size']
# MAX_TOKENS = config['max_tokens']

In [49]:
# num_layers = 1
# d_model = 110
# dff = 1024
# num_heads = 5
# dropout_rate = 0.15
# batch_size = 1024
# MAX_TOKENS = 65

In [50]:
# bs512_225_d_model256_dff_1024
num_layers = 1
d_model = 128
dff = 512
num_heads = 5
dropout_rate = 0.15
batch_size = 1024
MAX_TOKENS = 65

In [51]:
transformer = Transformer(
    # encoder_num_layers=encoder_num_layers,
    # decoder_num_layers=decoder_num_layers,
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer.get_vocab_size().numpy(),
    target_vocab_size=tokenizer.get_vocab_size().numpy(),
    dropout_rate=dropout_rate)

In [52]:
# checkpoint_dir = f"/home/vahan/Documents/machine_translation/models/d_model{config['d_model']}_dff{config['dff']}_numhead{config['num_head']}_numlayer1_bs{config['batch_size']}/ckpt/cp.ckpt"  
# print(checkpoint_dir)

In [53]:
checkpoint_dir = "/home/vahan/Documents/machine_translation/bs512_115_d_model128_dff_512_maxtoken_65/cp.ckpt"


In [54]:
transformer.load_weights(checkpoint_dir)

In [55]:
transformer_model = transformer.build_model(input_shape=(None,), target_shape=(None,))
transformer_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 context_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 target_input (InputLayer)   [(None, None)]               0         []                            
                                                                                                  
 encoder_2 (Encoder)         (None, None, 128)            4195200   ['context_input[0][0]']       
                                                                                                  
 decoder_2 (Decoder)         (None, None, 128)            4525184   ['target_input[0][0]',        
                                                                     'encoder_2[0][0]']     

In [56]:
class Corrector(tf.Module):
    def __init__(self, tokenizer, transformer):
        self.tokenizer = tokenizer
        self.transformer = transformer

    def __call__(self, sentence, max_length=MAX_TOKENS):
        # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
        assert isinstance(sentence, tf.Tensor)
        if len(sentence.shape) == 0:
            sentence = sentence[tf.newaxis]

        sentence = self.tokenizer.tokenize(sentence).to_tensor()

        encoder_input = sentence

        # As the output language is English, initialize the output with the
        # English `[START]` token.
        start_end = self.tokenizer.tokenize([''])[0]
        start = start_end[0][tf.newaxis]
        end = start_end[1][tf.newaxis]

        # `tf.TensorArray` is required here (instead of a Python list), so that the
        # dynamic-loop can be traced by `tf.function`.
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
        output_array = output_array.write(0, start)

        for i in tf.range(max_length):
            output = tf.transpose(output_array.stack())
            predictions = self.transformer([encoder_input, output], training=False)

            # Select the last token from the `seq_len` dimension.
            predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

            predicted_id = tf.argmax(predictions, axis=-1)

            # Concatenate the `predicted_id` to the output which is given to the
            # decoder as its input.

            output_array = output_array.write(i+1, predicted_id[0])

            if predicted_id == end:
                break

        output = tf.transpose(output_array.stack())
        # The output shape is `(1, tokens)`.
        text = tokenizer.detokenize(output)[0]  # Shape: `()`.
        # text = text.replace("եւ", "և")
         

        tokens = tokenizer.lookup(output)[0]



        # `tf.function` prevents us from using the attention_weights that were
        # calculated on the last iteration of the loop.
        # So, recalculate them outside the loop.
        self.transformer([encoder_input, output[:,:-1]], training=False)
        attention_weights = self.transformer.decoder.last_attn_scores

        return text, tokens, attention_weights

In [57]:
translator = Corrector(tokenizer, transformer)

In [58]:
def print_correction(sentence, tokens, ground_truth):
    print(f'{"Input:":15s}: {sentence}')
    print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
    print(f'{"Ground truth":15s}: {ground_truth}')

In [59]:
import re

class Preprocessing:
    def __init__(self, normalized_transcription, char_set = None, list_of_words = None):
        self.normalized_transcription = normalized_transcription
        #         self.new_sentences = set()
        self.new_sentences = list()
    def performe_preprocessing(self):
        pattern = r'.*\d.*'
        new_data = [sentence for sentence in self.normalized_transcription if not re.match(pattern, sentence)]
        self.normalized_transcription = new_data
        if " " in self.normalized_transcription:
            self.normalized_transcription.remove(" ")
        for sent in self.normalized_transcription:
            sent = sent.lower()
            new_sent = ""
            for i in sent:
                if ord(i) in range(ord("ա"), ord("և")+1) or i in {",", "։", " ", ":", ","}:
                    new_sent += i

            new_sent = new_sent.lower()

            if len(new_sent.strip())>1:
                if ":" in new_sent:
                    new_sent = new_sent.replace(":", "։")
                if new_sent[-1] == "։":
                    new_sent = new_sent.replace("։", "")
                new_sent = new_sent.replace("եվ", "և")
                new_sent = new_sent.replace("։", " ։")
                new_sent = new_sent.replace(",", ",")
                
                new_sent = new_sent.replace(",", " ,")
                new_sent = new_sent.replace(",", ", ")
                new_sent = " ".join(new_sent.split())

                self.new_sentences.append(new_sent)
        return self.new_sentences

In [65]:
sentence = "երկու հազար քսան թվանը կարղ է լինլ հեղափոխակա տաի"

ground_truth = ""

print("բառերի քանակը", len(sentence.split()))
print("տոկենների քանակը", tokenizer.tokenize(tf.constant([sentence]))[0].shape[0])

prp = Preprocessing(normalized_transcription=[sentence])
filtered_sentence = prp.performe_preprocessing()

start_time = time.time()

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(filtered_sentence))


end_time = time.time()
inference_time = end_time - start_time
print(f"Inference time: {inference_time:.4f} seconds")

print_correction(sentence, translated_text, ground_truth)

բառերի քանակը 9
տոկենների քանակը 17
Inference time: 1.0345 seconds
Input:         : երկու հազար քսան թվանը կարղ է լինլ հեղափոխակա տաի
Prediction     : երկու հազար քսան թվականը կարող է լինել հեղափոխական դասի
Ground truth   : 


In [ ]:
# import seaborn as sns
# import random
# import pickle
# import tensorflow as tf

In [ ]:
# def duplicate_words(sentence):
#     words = sentence.split()
#     num_words = len(words)
#     if num_words <= 1:
#         return sentence
# 
#     
#     min_duplicates = int(0.05*num_words)
#     max_duplicates = int(0.1*num_words)
#     num_duplicates = random.randint(min_duplicates, max_duplicates)
#     print(num_duplicates)
# 
#     duplicate_indices = random.sample(range(num_words), num_duplicates)
#     print(duplicate_indices)
#     for idx in sorted(duplicate_indices, reverse=True):
#         words.insert(idx, words[idx])
# 
#     return ' '.join(words)

In [ ]:
# sentence = "Ես գնում եմ տուն և ցանկանում եմ գնալ նաև բնակաչրան"
# len(sentence.split())